# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.233603464674                   -0.50    7.0
  2   -7.250055545646       -1.78       -1.40    1.0
  3   -7.250754713509       -3.16       -1.75    4.0
  4   -7.250916728470       -3.79       -1.85    2.0
  5   -7.251334512092       -3.38       -2.79    2.0
  6   -7.251338274984       -5.42       -3.19    3.0
  7   -7.251338775493       -6.30       -3.95    2.0
  8   -7.251338791960       -7.78       -4.14    4.0
  9   -7.251338798419       -8.19       -4.77    2.0
 10   -7.251338798655       -9.63       -5.22    3.0
 11   -7.251338798696      -10.38       -5.64    3.0
 12   -7.251338798704      -11.11       -6.27    3.0
 13   -7.251338798705      -12.74       -6.77    2.0
 14   -7.251338798705      -13.53       -7.29    3.0
 15   -7.251338798705      -14.45       -7.81    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04792449465197779
[ Info: Arnoldi iteration step 2: normres = 0.5964530614693303
[ Info: Arnoldi iteration step 3: normres = 0.9271443434480896
[ Info: Arnoldi iteration step 4: normres = 0.2277484128299124
[ Info: Arnoldi iteration step 5: normres = 0.5765231006741814
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.17e-02, 8.09e-02, 5.32e-01, 2.02e-01, 3.51e-02)
[ Info: Arnoldi iteration step 6: normres = 0.22212587292552544
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (5.72e-03, 9.97e-02, 1.52e-01, 1.06e-01, 2.47e-02)
[ Info: Arnoldi iteration step 7: normres = 0.13502370065915822
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.68e-04, 1.22e-02, 1.79e-02, 1.01e-01, 7.99e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11093236049358848
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.83e-